In [ ]:
import torch
import numpy as np
from torchvision.models import resnet18
import os
import shutil
import torch
from torch.utils.tensorboard import SummaryWriter

from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader,  TensorDataset
import pickle
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils

import torch.optim as optim


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
# http://localhost:8888/notebooks/Documents/AI/scripts/picture_comparison_crop80.ipynbfrom tqdm import tqdm_notebook, tqdm

from scipy import misc
import imageio
import glob
from PIL import Image

import keras as keras
from keras_unet.utils import plot_patches
from keras_unet.utils import get_patches
from keras_unet.utils import reconstruct_from_patches


In [ ]:
from os import listdir
from os.path import isfile, join
#paths to folders with images and segmentations
mypath_img = r"C:\Users\adoro\Documents\AI\histology pig big/"
mypath = r"C:\Users\adoro\Documents\AI\SegmentationClass/"


filename = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
len(filename)

In [ ]:
filename

In [ ]:
class myDataset(torch.utils.data.Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.targets[idx]
        if self.transform:
            image = self.transform(image)
        # sample = {"image": image, "label": label}

        return image, label

In [ ]:
def plot_random_Images(images_arr):
  fig, axes = plt.subplots(1, 5, figsize=(20,20))
  axes = axes.flatten()
  rand_num = np.random.randint(0, len_dataset, size=5)
  for img, ax in zip(images_arr[rand_num], axes):
    # img = np.round(img/np.max(img))
    ax.imshow(img.astype(int))
  plt.tight_layout()
  plt.show()
    

In [ ]:
def validate(model,testloader,device):
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        labels=labels.squeeze(1)
        # print(predicted)
        # print(labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # print(correct / total)
  
  return correct / total  

In [ ]:
def get_patches_custom(img_arr, size=256, stride=256):
    """
    Takes single image or array of images and returns
    crops using sliding window method.
    If stride < size it will do overlapping.
    
    Args:
        img_arr (numpy.ndarray): [description]
        size (int, optional): [description]. Defaults to 256.
        stride (int, optional): [description]. Defaults to 256.
    
    Raises:
        ValueError: [description]
        ValueError: [description]
    
    Returns:
        numpy.ndarray: [description]
    """    
    # check size and stride
    if size % stride != 0:
        raise ValueError("size % stride must be equal 0")

    patches_list = []
    overlapping = 0
    if stride != size:
        overlapping = (size // stride) - 1

    if img_arr.ndim == 3:
        i_max = img_arr.shape[0] // stride - overlapping
        j_max = img_arr.shape[1] // stride - overlapping
        for i in range(i_max):
            for j in range(j_max):
                # print(i*stride, i*stride+size)
                # print(j*stride, j*stride+size)
                patches_list.append(
                    img_arr[
                        i * stride : i * stride + size,
                        j * stride : j * stride + size
                    ]
                )

    elif img_arr.ndim == 4:
        i_max = img_arr.shape[1] // stride - overlapping
        for im in img_arr:
            for i in range(i_max):
                for j in range(i_max):
                    # print(i*stride, i*stride+size)
                    # print(j*stride, j*stride+size)
                    patches_list.append(
                        im[
                            i * stride : i * stride + size,
                            j * stride : j * stride + size,
                        ]
                    )

    else:
        raise ValueError("img_arr.ndim must be equal 3 or 4")

    return np.stack(patches_list)

In [ ]:
model = resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.float()
#name of weights file
model.load_state_dict(torch.load('model_weights_bs100_crop80_1A0_5000_5000.pth', map_location=torch.device('cpu')))
model.eval()

device = torch.device("cpu")

In [ ]:
n_area = 0
area_part = np.zeros((1, len(filename)))
area_true = area_part.copy()

for image_path in filename[0:len(filename)]:
    path = mypath_img+image_path[0:len(image_path)-3]+'tiff'
    image_in = imageio.imread(path)
    # size=(w_size,h_size)
    image = image_in[:,:,0:3]
    w_size = image.shape[0]
    h_size = image.shape[1]
    
    path = mypath+image_path
    mask = imageio.imread(path)
#     mask[:,:,1] = 0*mask[:,:,1] ########################### deleted green zones
    mask[:,:,2] = 0*mask[:,:,2]

    if  w_size> h_size:
        image = np.swapaxes(image,0,1)
        tmp = w_size
        w_size = h_size
        h_size = tmp
        
    mask_l = mask[0:w_size-1,:,:]
    mask_r = mask[1:w_size,:,:]
    mask_u = mask[:,0:h_size-1,:]
    mask_d = mask[:,1:h_size,:]
    mask_contour = abs(mask_r[:,0:h_size-1,:]-mask_l[:,0:h_size-1,:])+abs(mask_d[0:w_size-1,:,:]-mask_u[0:w_size-1,:,:])    
    
    half_width = 3
    for k in range(-half_width, half_width):
        mask_contour[half_width:-half_width,:,:] =  mask_contour[half_width:-half_width,:,:]+mask_contour[half_width+k:-half_width+k,:,:]
        mask_contour[:, half_width:-half_width,:] =  mask_contour[:, half_width:-half_width,:]+mask_contour[:, half_width+k:-half_width+k,:]

    mask_contour = (255*abs(1-np.sign(mask_contour))).astype(int)
    mask_contour[:,:,1] = mask_contour[:,:,0]
    mask_contour[:,:,2] = mask_contour[:,:,0]
        
    area_true[0,n_area] = np.sum(np.sum(mask[:,:,0]/255,0),0)/mask.shape[0]/mask.shape[1]
    print(image_path)
    print("true area: ", str(area_true[0,n_area]))   
    
    crop_size = 80
    
    x_image = get_patches_custom(
        img_arr=image.astype(int), # required - array of images to be cropped
        size=crop_size, # default is 256
        stride=crop_size) # default is 256
    
    x_crops_image = torch.transpose(torch.from_numpy(x_image), 1, 3)/255
    del(x_image)

    len_dataset = x_crops_image.shape[0]
    y_crops = np.zeros((len_dataset, 1))

    testset = myDataset(data=x_crops_image.float(), targets=y_crops)
    test_loader = DataLoader(testset, batch_size = 5000, shuffle = False)
#     print('dataset is ready')
    
    ################################

    k=0
    predicted = np.zeros((len_dataset,1))
    labels_true = predicted.copy()
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        # print(images.shape)
        # print(labels.shape)
        outputs = model(images)
        val, pred = torch.max(outputs.data, dim=1)
        predicted[k*5000:(k+1)*5000] = pred.unsqueeze(1)
    #     labels_true[k]=labels.squeeze(1)
        k=k+1
    #     print(k)

    n_w = w_size//crop_size
    n_h = h_size//crop_size

    predicted_array = np.reshape(predicted, (n_w, n_h))
    
    for n in range(1, n_w-1):
        for m in range(1, n_h-1):
            if predicted_array[n-1,m-1]==0 and predicted_array[n-1,m]==0 and predicted_array[n,m-1]==0 and \
            predicted_array[n+1,m+1]==0 and  predicted_array[n+1,m]==0 and  predicted_array[n,m+1]==0 and \
            predicted_array[n-1,m+1]==0 and  predicted_array[n+1,m-1]==0 and   predicted_array[n,m]==1:

                 predicted_array[n,m]=0 

#             if predicted_array[n-1,m-1]==1 and predicted_array[n-1,m]==1 and predicted_array[n,m-1]==1 and \
#             predicted_array[n+1,m+1]==1 and  predicted_array[n+1,m]==1 and  predicted_array[n,m+1]==1 and \
#             predicted_array[n-1,m+1]==1 and  predicted_array[n+1,m-1]==1 and   predicted_array[n,m]==0:

#                  predicted_array[n,m]=1
    
    predicted_filtered = np.reshape(predicted_array, (len_dataset, 1))    
    
    area_part[0,n_area] = np.sum(predicted_filtered)/len(predicted_filtered)
    print("predicted area: ", area_part[0,n_area])

    x_crops = torch.transpose(x_crops_image, 1, 3).detach().numpy()
    x_predicted = x_crops.copy()
    print(x_predicted.shape)

    red_square = np.zeros(np.shape(x_predicted[0,:,:,:]))
    green_square = red_square.copy()

    red_square[:,:,0] = 255 + red_square[:,:,0]
    green_square[:,:,0] = 255 + green_square[:,:,0]


    for k in range(len_dataset):
        if  predicted_filtered[k]==1:
            x_predicted[k,:,:,0] = np.minimum(x_crops[k,:,:,0]+100, red_square[:,:,0])

    ##################################


    x_reconstructed = reconstruct_from_patches(
        img_arr=x_predicted, # required - array of cropped out images
        org_img_size=(w_size, h_size), # required - original size of the image
        stride=crop_size) # use only if stride is different from patch size

    print("x_reconstructed shape: ", str(x_reconstructed.shape))
    segmented_image = np.minimum(x_reconstructed[0, 0:-1,0:-1,:], mask_contour[:,:,0:3]) #[0, 0:256*31+1, 0:256*31+1,:])

    fig = plt.figure(figsize=(50,50)) 
    plt.axis('off')
    plt.imshow(segmented_image) #x_reconstructed[0]) #[0, 0:256*31+1, 0:256*31+1,:])
    plt.show()

    image_name = image_path[0:len(image_path)-5]
    #path where you can save figures
    path = str(r"C:\Users\adoro\Documents\AI\test/"+image_name+'_'+str(round(area_part[0,n_area],3))+'_crop80_5000_5000.png')
    fig.savefig(path,  bbox_inches='tight')
    n_area = n_area+1
    
    


In [ ]:
print(area_part.T)
print(area_true.T)